In [1]:
# Import libraries

import sys
import pypsa
import logging
import warnings
import pandas as pd
import numpy as np
import cartopy.crs as ccrs
from plotly import express as px
from plotly import graph_objects as go
import matplotlib.pyplot as plt
import ipywidgets as widgets
from pathlib import Path

root_path = Path(globals()['_dh'][0]).resolve().parent
sys.path.append(str(root_path))

import paths
from library.assumptions import read_assumptions
from library.demand import projected_energy
from library.weather import generate_cutout
from library.renewables import capacity_factor

logging.basicConfig(level=logging.INFO)

In [2]:
# Set the configuration

## Parameters you won't change very often
base_currency = 'SEK'
exchange_rates = {
    'EUR': 11.68,
    'USD': 10.70
}
base_year = 2024
discount_rate = 0.05
onwind_turbine =  "2030_5MW_onshore.yaml"
offwind_turbine = "2030_20MW_offshore.yaml"
resolution = 3

## Parameters that will change frequently
target_year = 2030
use_offwind = False
use_h2 = False
h2_initial = 0
biogas_limit = 0
load_target = 10

In [3]:
# Load the data needed from assumptions, the electricity demand, and the atlite output from ERA5 weather data for VGR 2023

## Transform assumptions to range base_year to target_year
assumptions = read_assumptions(paths.input_path / 'assumptions.csv', base_year, target_year, base_currency, exchange_rates, discount_rate)

# Read the normalized demand from csv file (see normalize_demand() in library.demand for details)
# And then calculate target_load using projection of energy need in target_year
normalized_demand = pd.read_csv(paths.input_path / 'demand/normalized-demand-2023-3h.csv', delimiter=',')
# target_load = projected_energy(target_year, 1.21265) * normalized_demand['se3'].values.flatten() * 1_000_000
target_load = load_target * normalized_demand['value'].values.flatten() * 1_000_000

# Create of load the cutout from atlite (we assume weather data from 2023 and a 3h window)
geo = '14' # All of VGR
cutout, selections, eez, index = generate_cutout(geo, None, '2023-01', '2023-12')
selection = selections[geo]

capacity_factor_solar = capacity_factor(cutout, selection, 'solar', '', geo, None, '2023-01', '2023-12').values.flatten()
capacity_factor_onwind = capacity_factor(cutout, selection, 'onwind', onwind_turbine, geo, None, '2023-01', '2023-12').values.flatten()
capacity_factor_offwind = capacity_factor(cutout, selection, 'offwind', offwind_turbine, geo, None, '2023-01', '2023-12').values.flatten()


/home/viktor/miniforge3/envs/pypsa-vgr/lib/python3.11/site-packages/atlite/cutout.py:191: UserWarning:

Arguments module, x, y, time, dx, dy, dt are ignored, since cutout is already built.

INFO:atlite.data:Storing temporary files in /tmp/tmpk41_9pzk
INFO:atlite.convert:Convert and aggregate 'pv'.
/home/viktor/miniforge3/envs/pypsa-vgr/lib/python3.11/site-packages/atlite/resource.py:72: FutureWarning:

'add_cutout_windspeed' for wind turbine
power curves will default to True in atlite relase v0.2.13.

INFO:atlite.convert:Convert and aggregate 'wind'.
/home/viktor/miniforge3/envs/pypsa-vgr/lib/python3.11/site-packages/atlite/resource.py:72: FutureWarning:

'add_cutout_windspeed' for wind turbine
power curves will default to True in atlite relase v0.2.13.

INFO:atlite.convert:Convert and aggregate 'wind'.


In [4]:
# Build the network

def annuity(r, n):
    return r / (1.0 - 1.0 / (1.0 + r) ** n)

def annualized_capex(asset):
    return (annuity(float(assumptions.loc[('general', 'discount_rate'), 'value']), float(assumptions.loc[(asset, 'lifetime'), 'value'])) + float(assumptions.loc[(asset, 'FOM'), 'value'])) * float(assumptions.loc[(asset, 'capital_cost'), 'value'])

## Initialize the network
network = pypsa.Network()
network.set_snapshots(index)
network.snapshot_weightings.loc[:, :] = resolution

## Carriers
carriers = [
    'AC',
    'onwind',
    'offwind',
    'solar',
    'li-ion',
    'h2',
    'biogas',
    'mixedgas',
    'backstop',
    ]

carrier_colors = ['black', 'green', 'blue', 'red', 'lightblue', 'grey', 'brown', 'brown', 'white']

network.madd(
    'Carrier',
    carriers,
    color=carrier_colors,
    )

## Load bus location
minx, miny, maxx, maxy = selection.total_bounds
midx = (minx + maxx)/2
midy = (miny + maxy)/2

## Add the buses
network.add('Bus', 'Load bus', carrier='AC', x=midx, y=midy)
network.add('Bus', 'Renewables bus', x=midx+0.5, y=midy+0.25)
network.add('Bus', 'Battery bus', carrier='li-ion', x=midx-0.5, y=midy)
if use_h2 or biogas_limit > 0:
    network.add('Bus', 'Gas turbine', x=midx, y=midy+0.5)
if use_h2:
    network.add('Bus', 'H2 bus', carrier='h2', x=midx-0.5, y=midy+0.5)
if biogas_limit > 0:
    network.add('Bus', 'Biogas market', x=midx, y=midy+0.9)


## Add load and backstop to load bus
network.add('Load', 'Desired load', bus='Load bus',
            p_set=target_load
            )

network.add('Generator', 'Backstop', carrier='backstop', bus='Load bus',
            p_nom_extendable=True,
            capital_cost=assumptions.loc[('backstop', 'capital_cost'), 'value'],
            marginal_cost=assumptions.loc[('backstop', 'marginal_cost'), 'value'],
            lifetime=assumptions.loc[('backstop', 'lifetime'), 'value'],
            )

## Add generators and links to renewable bus

network.add('Generator', 'Solar park', carrier='solar', bus='Renewables bus',
            p_nom_extendable=True, 
            p_max_pu=capacity_factor_solar,
            p_nom_mod=assumptions.loc['solar','unit_size'].value,
            capital_cost= annualized_capex('solar'),
            marginal_cost=assumptions.loc[('solar', 'VOM'), 'value'],
            lifetime=assumptions.loc[('solar', 'lifetime'), 'value'],
            )

network.add('Generator', 'Onwind park', carrier='onwind', bus='Renewables bus',
            p_nom_extendable=True,
            p_max_pu=capacity_factor_onwind,
            p_nom_mod=assumptions.loc['onwind','unit_size'].value,
            capital_cost= annualized_capex('onwind'),
            marginal_cost=assumptions.loc[('onwind', 'VOM'), 'value'],
            lifetime=assumptions.loc['onwind','lifetime'].value,
            )

if use_offwind:
    network.add('Generator', 'Offwind park', carrier='offwind', bus='Renewables bus',
                p_nom_extendable=use_offwind,
                p_max_pu=(capacity_factor_offwind if use_offwind else [0] * len(capacity_factor_offwind)),
                p_nom_mod=assumptions.loc['offwind','unit_size'].value,
                capital_cost= annualized_capex('offwind'),
                marginal_cost=assumptions.loc[('offwind', 'VOM'), 'value'],
                lifetime=assumptions.loc['offwind','lifetime'].value,
                )

network.add('Link', 'Renewables load link', bus0='Renewables bus', bus1='Load bus',
            p_nom_extendable=use_offwind,
            )

## Add battery storage

network.add('Link','Battery charge', bus0='Renewables bus', bus1='Battery bus',
            p_nom_extendable = True,
            capital_cost= annualized_capex('battery_inverter'),
            marginal_cost=assumptions.loc['battery_inverter','VOM'].value,
            lifetime=assumptions.loc['battery_inverter','lifetime'].value,
            efficiency = assumptions.loc['battery_inverter','efficiency'].value,
            )

network.add('Store', 'Battery storage', carrier='li-ion', bus='Battery bus',
            e_initial=100,
            e_nom_extendable=True,
            e_cyclic=True,
            e_min_pu=0.15,
            standing_loss=0.00008, # TODO: Check if this is really per hour as in the documentation or if it is per snapshot
            capital_cost= annualized_capex('battery_storage'),
            marginal_cost=assumptions.loc['battery_storage','VOM'].value,
            lifetime=assumptions.loc['battery_storage', 'lifetime'].value,
            )

network.add('Link','Battery discharge', carrier='li-ion', bus0='Battery bus', bus1='Load bus',
            p_nom_extendable = True,
            efficiency = assumptions.loc['battery_inverter','efficiency'].value,
            )

## Add H2 electrolysis, storage, pipline to gas turbine

if use_h2:
    network.add('Link', 'H2 electrolysis', carrier='h2', bus0='Renewables bus', bus1='H2 bus',
                p_nom_extendable=True,
                p_nom_mod=assumptions.loc['h2_electrolysis','unit_size'].value,
                capital_cost= annualized_capex('h2_electrolysis'),
                marginal_cost=assumptions.loc[('h2_electrolysis', 'VOM'), 'value'],
                lifetime=assumptions.loc['h2_electrolysis','lifetime'].value,
                efficiency=assumptions.loc['h2_electrolysis','efficiency'].value,
                )

    network.add('Store', 'H2 storage', carrier='h2', bus='H2 bus',
                e_initial=(150_000 if use_h2 else 0),
                e_nom_extendable=use_h2,
                e_cyclic=True,
                capital_cost= annualized_capex('h2_storage'),
                marginal_cost=assumptions.loc['h2_storage','VOM'].value,
                lifetime=assumptions.loc['h2_storage','lifetime'].value
                )

    network.add('Link', 'H2 pipeline', carrier='h2', bus0='H2 bus', bus1='Gas turbine',
                p_nom_extendable=True,
                )

### Biogas pipeline

if biogas_limit > 0:
    network.add('Generator', 'Biogas input', carrier='biogas', bus='Biogas market',
                p_nom_extendable=True,
                p_nom_max=biogas_limit,
                marginal_cost=assumptions.loc['biogas','cost'].value,
                lifetime=100,
                )

    network.add('Link', 'Biogas pipeline', carrier='biogas', bus0='Biogas market', bus1='Gas turbine',
                p_nom_extendable=True,
                )

### Gas turbines
if use_h2 or biogas_limit > 0:
         
    network.add('Link', 'Combined Cycle Gas turbine', carrier='mixedgas', bus0='Gas turbine', bus1='Load bus',
                p_nom_extendable=True,
                p_nom_mod=assumptions.loc['combined_cycle_gas_turbine','unit_size'].value,
                capital_cost= annualized_capex('combined_cycle_gas_turbine'),
                marginal_cost=assumptions.loc['combined_cycle_gas_turbine','VOM'].value,
                lifetime=assumptions.loc['combined_cycle_gas_turbine','lifetime'].value,
                efficiency=assumptions.loc['combined_cycle_gas_turbine','efficiency'].value,
                )

In [5]:
# Add constraints to the model and run the optimization

## Create the model
model = network.optimize.create_model()

generator_capacity = model.variables["Generator-p_nom"]
link_capacity = model.variables["Link-p_nom"]

## Add offwind constraint
if use_offwind:
    offwind_percentage = 0.5

    offwind_constraint = (1 - offwind_percentage) / offwind_percentage * generator_capacity.loc['Offwind park'] - generator_capacity.loc['Onwind park']
    model.add_constraints(offwind_constraint == 0, name="Offwind_constraint")

## Add battery charge/discharge ratio constraint
lhs = link_capacity.loc["Battery charge"] - network.links.at["Battery charge", "efficiency"] * link_capacity.loc["Battery discharge"]
model.add_constraints(lhs == 0, name="Link-battery_fix_ratio")

## Run optimization
network.optimize.solve_model(solver_name='highs')

INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing integer variables.: 100%|██████████| 1/1 [00:00<00:00, 760.11it/s]
INFO:linopy.io: Writing time: 0.26s
INFO:linopy.solvers:Log file at /tmp/highs.log


Running HiGHS 1.7.2 (git hash: 184e327): Copyright (c) 2024 HiGHS under MIT licence terms


INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 23368 primals, 52569 duals
Objective: 1.74e+10
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-ext-p-lower, Generator-ext-p-upper, Link-fix-p-lower, Link-fix-p-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, Store-energy_balance were not assigned to the network.


Coefficient ranges:
  Matrix [4e-06, 5e+00]
  Cost   [8e+01, 1e+06]
  Bound  [0e+00, 0e+00]
  RHS    [7e+02, 2e+03]
Presolving model
23372 rows, 14616 cols, 54050 nonzeros  0s
17776 rows, 9020 cols, 51605 nonzeros  0s
17306 rows, 8550 cols, 53482 nonzeros  0s

Solving MIP model with:
   17306 rows
   8550 cols (0 binary, 2 integer, 0 implied int., 8548 continuous)
   53482 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   -inf            inf                  inf        0      0      0         0     0.1s
 S       0       0         0   0.00%   -inf            17403308161.15     Large        0      0      0         0     0.5s
 R       0       0         0   0.00%   17401511262.94  17401515726.63     0.00%        0      0      0     11570     0.5s

So

('ok', 'optimal')

In [7]:
network.statistics()

Optimal Capacity  Installed Capacity        Supply  \
Generator backstop       1686.392061                 0.0  4.455970e+05   
          onwind         4663.362857                 0.0  5.476981e+06   
          solar          4824.257340                 0.0  4.912774e+06   
Link      AC             2622.108901                 0.0  9.965499e+06   
          li-ion         2733.738374                 0.0  9.554403e+06   
Load      -                 0.000000                 0.0  0.000000e+00   
Store     li-ion        16659.511116                 0.0  2.118789e+06   

                      Withdrawal      Dispatch  Transmission  Capacity Factor  \
Generator backstop  1.880575e-10  4.455970e+05           0.0         0.030163   
          onwind    0.000000e+00  5.476981e+06           0.0         0.134072   
          solar     0.000000e+00  4.912774e+06           0.0         0.116250   
Link      AC        1.038975e+07 -4.242552e+05           0.0         0.452325   
          li-ion    9.961157e+06 -4.067539e+05           0.0         0.415957   
Load      -         1.000000e+07 -1.000000e+07           0.0              NaN   
Store     li-ion    2.123131e+06 -4.342512e+03           0.0         0.371980   

                     Curtailment  Capital Expenditure  \
Generator backstop  0.000000e+00         0.000000e+00   
          onwind    1.015839e+07         6.286029e+09   
          solar     5.370525e+05         2.505845e+09   
Link      AC        0.000000e+00         5.595347e+08   
          li-ion    0.000000e+00         0.000000e+00   
Load      -         0.000000e+00         0.000000e+00   
Store     li-ion    0.000000e+00         3.208489e+09   

                    Operational Expenditure  Revenue  Market Value  
Generator backstop             4.684115e+09      0.0           NaN  
          onwind               1.576252e+08      0.0           NaN  
          solar                0.000000e+00      0.0           NaN  
Link      AC                   0.000000e+00      0.0           NaN  
          li-ion               0.000000e+00      0.0           NaN  
Load      -                    0.000000e+00      0.0           0.0  
Store     li-ion              -1.218226e+05      0.0           NaN